Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [2]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [3]:
import funciones

In [4]:
#anio = int(input('Ingrese el año: '))
#mes = int(input('Ingrese el mes: '))

anio = 2024
mes = 1

In [5]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [6]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [7]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    sql_command01 = f"EXEC sp_get_indice_grupo {anio}, {mes}"
    cursor.execute(sql_command01)
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [31]:
#boletas

___________________________________________________________________________________________

In [9]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [24]:
# Creamos un dataframe con las columnas que se van a utilizar
# Selecciona las columnas que se van a utilizar
bol = boletas[boletas['tp_poll_impute_status'].isin([351821, 351824])]
bol = boletas[boletas['detail_tp_poll_status'].isin([351634, 351635])]
bol = bol[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']]
bol.insert(1, 'cod_prod', np.where(bol['codigo_articulo'].str.len() > 8, 
                                   bol['codigo_articulo'].str.slice(0, -2), 
                                   bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = bol['cantidad_anterior'].astype(float)
bol['cantidad_actual'] = bol['cantidad_actual'].astype(float)
bol['precio_anterior'] = bol['precio_anterior'].astype(float)
bol['precio_actual'] = bol['precio_actual'].astype(float)
bol['cantidad_anterior'] = bol.apply(lambda row: row['cantidad_actual'] if row['cantidad_anterior'] == 0 or pd.isnull(row['cantidad_anterior']) else row['cantidad_anterior'], axis=1)
bol['precio_anterior'] = bol.apply(lambda row: row['precio_actual'] if row['precio_anterior'] == 0 or pd.isnull(row['precio_anterior']) else row['precio_anterior'], axis=1)

bol

,region,cod_prod,codigo_articulo,articulo,cantidad_anterior,cantidad_actual,precio_anterior,precio_actual,nt_tipo
0,1,0540301,5403012,Olla y cacerola de 1 a 2lt,1.0,1.0,145.0,145.0,NaN
1,1,0312306,3123061,Vestidos formales con mangas de largo hasta la...,1.0,1.0,250.0,250.0,NaN
2,1,0721101,7211011,Llantas para automóvil 185/65 R14,1.0,1.0,325.0,400.0,NaN
3,1,0411101,4111013,Alquiler de apartamento (con o sin servicios),1.0,1.0,1700.0,1700.0,NaN
4,1,0722401,7224013,Aceite de motor sintético,946.0,946.0,70.0,70.0,NaN
...,...,...,...,...,...,...,...,...,...
51522,8,0540101,5401011,Vajilla de porcelana/cerámica/vidrio para 4 pe...,1.0,1.0,115.0,115.0,NaN
51523,8,0540304,5403041,Vasos de plástico y melamina,1.0,1.0,1.0,1.0,NaN
51524,8,0561102,5611022,Bola (barra) de jabón para ropa (unidad),350.0,350.0,7.5,7.5,NaN
51525,8,0561906,5619061,Vela o candela blanca (unidad o paquete),1.0,1.0,7.0,7.0,NaN


In [41]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
pon_prod = funciones.pondcat('Producto', ponderaciones).rename(columns={'grupo_codigo':'cod_prod'})
pon_prod['cod_prod'] = pon_prod['cod_prod'].apply(lambda x: '0' + x if len(str(x)) == 6 else x)
pon_prod

,region,cod_prod,grupo_nombre,ponderacion_region,indice_anterior
4,0,0111101,Arroz,0.522366964214768000,100.000000
5,0,0111102,Maíz,0.711087208966818000,100.000000
7,0,0111201,Harina de trigo,0.055904629092940400,100.000000
8,0,0111202,Harina de maíz,0.170135799350814000,100.000000
9,0,0111203,Harina para atoles,0.143783212432974000,100.000000
...,...,...,...,...,...
6817,8,1390901,Servicios legales y financieros,0.194021898126118000,100.000000
6818,8,1390902,Servicios funerarios,0.037005094003931200,100.000000
6819,8,1390903,Servicios de registro civil,0.056273352029694800,100.000000
6820,8,1390904,Pago de impuestos de circulación de vehículo,0.146902379060297000,100.000000


Calculo del IPC

In [13]:
#funciones que luego voy a pasar al archivo que tiene todas las funciones
def variacion(x):
    x['var_prod'] = (x['precio_actual'] / x['cantidad_actual']) * (x['cantidad_anterior'] / x['precio_anterior'])
    return x

In [14]:
# Se calcula la variacion de las variedades
variacion = variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]

#bol['var_prod'].isnull().sum()

C:\Users\sncap\AppData\Local\Temp\ipykernel_41376\1487776982.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['var_prod'] = (x['precio_actual'] / x['cantidad_actual']) * (x['cantidad_anterior'] / x['precio_anterior'])
C:\Users\sncap\AppData\Local\Temp\ipykernel_41376\385769678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)


In [15]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())

relativo

,region,cod_prod,relativo
0,1,0111101,NaN
1,1,0111102,0.984308
2,1,0111201,NaN
3,1,0111202,NaN
4,1,0111203,1.025552
...,...,...,...
2648,8,1321101,1.000000
2649,8,1321104,1.000000
2650,8,1329102,1.000000
2651,8,1390901,1.000000


In [16]:
# Step 1: Calculate geometric mean by grouping region and cod_prod
relativo = (bol01.groupby(['region', 'cod_prod'])
                  .agg(relativo=('var_prod', lambda x: gmean(x)))
                  .reset_index())

# Step 2: Left join with pon_prod DataFrame
relativo = (relativo.merge(pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']], 
                           on=['region', 'cod_prod'], 
                           how='left')
                    .rename(columns={'relativo': 'geometric_mean'}))

# Step 3: Select required columns
relativo = relativo[['region', 'cod_prod', 'grupo_nombre', 'geometric_mean', 'ponderacion_region', 'indice_anterior']]

NameError: name 'bol01' is not defined